In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os
import numpy as np
from datetime import datetime
from collections  import Counter
from nltk import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from wordcloud import WordCloud ,STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
#print(os.listdir("../input"))
from gensim.models import word2vec
from sklearn.manifold import TSNE
from nltk import pos_tag
from nltk.help import upenn_tagset
import gensim
import matplotlib.colors as mcolors
from nltk import jaccard_distance
from nltk import ngrams
#import textstat
plt.style.use('ggplot')

In [12]:
df1 = pd.read_csv('Data/Additional data/kaggle_data_dictionary.csv')

In [46]:
df1

,Field Name,Annotation Letter,Description,Data Type,Allowable Values,Accepts Null Values?,Additional Notes
0,FILE_NAME,NaN,The file name of the job bulletin from which e...,String,NaN,No,NaN
1,JOB_CLASS_TITLE,A,"The title of the particular class (e.g., Syste...",String,Any job title matching in in supplied job bull...,No,NaN
2,JOB_CLASS_NO,B,The 4-digit code that corresponds to the job c...,String,Any four character numeric string matching one...,No,Some classes have three non-zero digits — e.g....
3,REQUIREMENT_SET_ID,D1,A main requirement that has to be filled. Thes...,Integer,Any positive integer,No,NaN
4,REQUIREMENT_SUBSET_ID,D2,"A secondary requirement that has to be filled,...",String,Any letter or combination of letters.,No,"A-Z represents 1-26. In the case of 27th, 28th..."
5,JOB_DUTIES,C,A summary of what someone does in the particul...,String,NaN,No,NaN
6,EDUCATION_YEARS,F,Years required in a particular education program,Float,Any positive float representing years or a fra...,Yes,Any job description that mentions months shoul...
7,SCHOOL_TYPE,G,School Type: School type required (e.g. colleg...,String,"COLLEGE OR UNIVERSITY, HIGH SCHOOL, APPRENTICE...",Yes,Allowable value list is most likely incomplete...
8,EDUCATION_MAJOR,H,Major or concentration required,String,NaN,Yes,A list of many sufficient majors should appear...
9,EXPERIENCE_LENGTH,I,Years required in a particular job class or ex...,Float,NaN,Yes,Some classes may specify months rather than ye...


In [39]:
def read_txt(file):
    f = open(file, "r")
    return f.read()
    
    

In [40]:
ACCOUNTING_CLERK =read_txt("Data/Job Bulletins/ACCOUNTING CLERK 1223 071318.txt")

In [41]:
ACCOUNTING_CLERK

'ACCOUNTING CLERK\n\nClass Code:       1223\nOpen Date:  07-13-18\n(Exam Open to All, including Current City Employees)\n\nANNUAL SALARY\n\n$49,005 to $71,618\n\nNOTE:\n\nAnnual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.\n\nDUTIES\n\nAn Accounting Clerk performs difficult and responsible clerical accounting work in the review, preparation, processing, reconciliation, and maintenance of accounting records and reports, or payroll and time keeping documents; processing and monitoring of invoice payments, and personal services contract invoice payments; may perform cashiering duties; or may act as a lead person in the performance of such work.\n\nREQUIREMENT/MINIMUM QUALIFICATION\n\nTwo years of full-time paid office clerical experience, including one year of full-time paid clerical accounting work.\n\nSuccessful completion of 6 semester or 9 quarter unit

In [45]:
splited_text = ACCOUNTING_CLERK.split('\n')

In [48]:
splited_text

['ACCOUNTING CLERK',
 '',
 'Class Code:       1223',
 'Open Date:  07-13-18',
 '(Exam Open to All, including Current City Employees)',
 '',
 'ANNUAL SALARY',
 '',
 '$49,005 to $71,618',
 '',
 'NOTE:',
 '',
 'Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.',
 '',
 'DUTIES',
 '',
 'An Accounting Clerk performs difficult and responsible clerical accounting work in the review, preparation, processing, reconciliation, and maintenance of accounting records and reports, or payroll and time keeping documents; processing and monitoring of invoice payments, and personal services contract invoice payments; may perform cashiering duties; or may act as a lead person in the performance of such work.',
 '',
 'REQUIREMENT/MINIMUM QUALIFICATION',
 '',
 'Two years of full-time paid office clerical experience, including one year of full-time paid clerical accounting w

In [49]:
no_space_list = [item.remove('') for item in splited_text]

AttributeError: 'str' object has no attribute 'remove'

In [34]:
lst1 = []
lst2 = []
for entry in ACCOUNTING_CLERK:
    if entry.isupper() == True:
        lst1.append(entry)

In [26]:
lst1

['A',
 'C',
 'C',
 'O',
 'U',
 'N',
 'T',
 'I',
 'N',
 'G',
 'C',
 'L',
 'E',
 'R',
 'K',
 'C',
 'C',
 'O',
 'D',
 'E',
 'O',
 'A',
 'C',
 'C',
 'E',
 'A',
 'N',
 'N',
 'U',
 'A',
 'L',
 'S',
 'A',
 'L',
 'A',
 'R',
 'Y',
 'N',
 'O',
 'T',
 'E',
 'A',
 'T',
 'P',
 'D',
 'U',
 'T',
 'I',
 'E',
 'S',
 'A',
 'A',
 'C',
 'R',
 'E',
 'Q',
 'U',
 'I',
 'R',
 'E',
 'M',
 'E',
 'N',
 'T',
 'M',
 'I',
 'N',
 'I',
 'M',
 'U',
 'M',
 'Q',
 'U',
 'A',
 'L',
 'I',
 'F',
 'I',
 'C',
 'A',
 'T',
 'I',
 'O',
 'N',
 'T',
 'S',
 'P',
 'R',
 'O',
 'C',
 'E',
 'S',
 'S',
 'N',
 'O',
 'T',
 'E',
 'S',
 'A',
 'H',
 'A',
 'S',
 'Q',
 'A',
 'A',
 'S',
 'Q',
 'S',
 'A',
 'P',
 'C',
 'L',
 'A',
 'A',
 'A',
 'I',
 'C',
 'P',
 'D',
 'A',
 'A',
 'R',
 'W',
 'H',
 'E',
 'R',
 'E',
 'T',
 'O',
 'A',
 'P',
 'P',
 'L',
 'Y',
 'A',
 'A',
 'D',
 'W',
 'A',
 'O',
 'N',
 'O',
 'T',
 'E',
 'A',
 'A',
 'P',
 'P',
 'L',
 'I',
 'C',
 'A',
 'T',
 'I',
 'O',
 'N',
 'D',
 'E',
 'A',
 'D',
 'L',
 'I',
 'N',
 'E',
 'A',
 'F',
 'F'